In [1]:
import pandas as pd
import json
from itertools import chain

files = [
    "js_2023_02_06/warm/query_responses.json",
    "js_2023_02_06/cold/query_responses.json",
    "merlin/cmi-cache-only/all-commands/main/full_responses.json",
    "merlin/cmi-cache-only/all-commands/merlin-domains/full_responses.json",
    # "merlin/cmi-cache-only/complete-prefix/main/full_responses.json",
    # "merlin/cmi-cache-only/complete-prefix/merlin-domains/full_responses.json",
    "merlin/full-cache/all-commands/main/full_responses.json",
    "merlin/full-cache/all-commands/merlin-domains/full_responses.json"
]


def frame_from_file(file):
    data = pd.read_json(file, lines = True)

    resp = list(chain.from_iterable(data['responses']))

    resp_with_id = []

    for _, row in data.iterrows():
        sample_id = row['sample_id']
        unix_time = row['unix_time']
        for resp in row['responses']:
            resp_copy = resp.copy()
            resp_copy['sample_id'] = sample_id
            resp_copy['unix_time'] = unix_time
            resp_with_id.append(resp_copy)

    return pd.json_normalize(resp_with_id, sep='.')


In [2]:
data = pd.read_json("merlin/full-cache/all-commands/merlin-domains/full_responses.json", lines = True)

resp = list(chain.from_iterable(data['responses']))
resp_with_id = []
for _, row in data.iterrows():
    sample_id = row['sample_id']
    for resp in row['responses']:
        resp_copy = resp.copy()
        resp_copy['sample_id'] = sample_id
        resp_with_id.append(resp_copy)
pd.json_normalize(resp_with_id, sep='.')

,class,value,notifications,heap_mbytes,query_num,sample_id,timing.clock,timing.cpu,timing.query,timing.pp,...,cache.cmt.hit,cache.cmt.miss,cache.cmi.hit,cache.cmi.miss,value.file,value.pos.line,value.pos.col,value.entries,value.context,cache.typer
0,return,[],[],4480,0,4238,16,16,2,0.0,...,0.0,0.0,27.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,return,NaN,[],4479,0,4237,17,17,3,0.0,...,0.0,0.0,27.0,0.0,std.ml,361.0,11.0,NaN,NaN,NaN
2,return,NaN,[],4477,0,4236,17,17,3,0.0,...,2.0,0.0,27.0,0.0,stdlib.ml,83.0,9.0,NaN,NaN,NaN
3,return,NaN,[],4475,0,4235,19,19,3,0.0,...,0.0,0.0,27.0,0.0,NaN,NaN,NaN,"[{'name': 's', 'kind': 'Value', 'desc': '', 'i...",None,NaN
4,return,NaN,[],4473,0,4234,18,18,3,0.0,...,0.0,0.0,27.0,0.0,NaN,NaN,NaN,"[{'name': '()', 'kind': 'Constructor', 'desc':...",None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,return,"[{'start': {'line': 106, 'col': 6}, 'end': {'l...",[],60,0,4,10,12,1,0.0,...,84.0,0.0,34.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3995,return,"[{'start': {'line': 85, 'col': 45}, 'end': {'l...",[],58,0,3,10,13,11,0.0,...,62.0,0.0,34.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3996,return,"[{'start': {'line': 76, 'col': 53}, 'end': {'l...",[],56,0,2,11,14,13,0.0,...,139.0,29.0,68.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN
3997,error,Destruct not allowed on arrow type,[],54,0,1,29,40,40,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def print_percentiles(frame, file):
    percentiles = [0.25, 0.5, 0.75, 0.95]


    desc_clock = frame['timing.clock'].describe(percentiles=percentiles)
    desc_cpu = frame['timing.cpu'].describe(percentiles=percentiles)
    desc_reader = frame['timing.reader'].describe(percentiles=percentiles)
    desc_ppx = frame['timing.ppx'].describe(percentiles=percentiles)
    desc_typer = frame['timing.typer'].describe(percentiles=percentiles)
    desc_query = frame['timing.query'].describe(percentiles=percentiles)
    # desc_unix = frame['unix_time'].describe(percentiles=percentiles)


    df_clock  = pd.DataFrame(desc_clock).T
    df_cpu    = pd.DataFrame(desc_cpu).T
    df_reader = pd.DataFrame(desc_reader).T
    df_ppx    = pd.DataFrame(desc_ppx).T
    df_typer  = pd.DataFrame(desc_typer).T
    df_query  = pd.DataFrame(desc_query).T
    # df_unix   = pd.DataFrame(desc_unix).T


    df_clock.index  = ["timing.clock"]
    df_cpu.index    = ["timing.cpu"]
    df_reader.index = ["timing.reader"]
    df_ppx.index    = ["timing.ppx"]
    df_typer.index  = ["timing.typer"]
    df_query.index  = ["timing.query"]
    # df_unix.index   = ["unix_time"]


    parts = file.split("/")
    cache_part = parts[1]
    cmd_part = parts[2]
    branch_part = parts[3]
    print(f"Branch: {branch_part}")
    print(f"Cache: {cache_part}")
    print(f"Commands: {cmd_part}")


    display(pd.concat([ df_clock, df_cpu, df_reader, df_ppx, df_typer, df_query]))
    print("\n\n")

for f in files:
    frame = frame_from_file(f)
    print_percentiles(frame, f)

KeyError: 'unix_time'

In [3]:
sorted = comp_times.sort_values(ascending=False)

sorted.head(20)

NameError: name 'comp_times' is not defined